# 사용자 댓글을 이용한 네이버 기사 추천

### 가정 : 사용자 댓글을 분석하면 그 사용자에게 맞는 기사를 추천이 가능할 것이다.

- Komoran 형태소 분석기를 이용해서 기사와 댓글내용의 고유명사와 일반명사만 추출 하여 vectorized 한다.
- cos 거리 측정을 이용하여 댓글과 각 기사의 거리를 측정하여 가장 가까운 기사를 추천한다.

##### Import Package

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import konlpy.tag 
import pandas as pd
import scipy as sp
from scipy import spatial
import operator
import numpy as np

##### Get Article & Comment

In [2]:
def shuffle_df(df):
    """shuffle pandas dataframe"""
    
    return df.iloc[np.random.permutation(len(df))].reset_index(drop=True) 

def get_datas(date, comment=1):
    """ get naver news articles and comments"""
    
    article_df = pd.read_csv("../data/news/" + date + ".csv")
#     article_df = shuffle_df(article_df)
    comment_df = pd.read_csv("../data/comment/comment"+str(comment)+".csv")
    
    return article_df, comment_df

#####  Morpheme Analytics

In [3]:
def make_document_morpheme(texts):
    """make documents list after morphological"""
    
    documents = []
    
    komoran = konlpy.tag.Komoran()

    for text in texts:
        obj = line2obj(komoran.pos(text, flatten=False))
        documents.append(obj)
        
    return documents

def line2obj(lines):
    """ make keywords dictionary include only (NNP, NNG)"""
    
    obj = {}
    
    for line in lines:
        for keyword in line:
            if len(keyword[0]) > 1 and ( keyword[1] == "NNP" or keyword[1] == "NNG" ) :
                key = keyword[0]
                if key in obj:
                    obj[key] += 1
                else:
                    obj[key] = 1
                    
    return obj

In [4]:
def gen_text_documents(documents):
    
    text_documents = []
    
    for document in documents:
        
        text_words = []
        
        for word, count in document.items():
            text_words.extend([word] * count)
            
        text_document = " ".join(text_words)
        text_documents.append(text_document)
        
    return text_documents        

In [5]:
def calc_euc_dist(v1, v2):
    """ euclidean distance """
    
    delta = v1 - v2
    dist = sp.linalg.norm(delta.toarray())
    return dist

def calc_cos_dist(v1, v2):
    """ cosin distance """
#     v1 = v1.toarray()
#     v2 = v2.toarray()
#     dist = 1.0 - np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    dist = spatial.distance.cosine(v1.toarray(), v2.toarray())
    
    return dist

def get_euc_dists(vectorized, num_docs, comm="euc"):
    
    dists = []
    
    for idx in range(num_docs-1):
            
        if comm == "euc":
            dist = calc_euc_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
            dists.append((num_docs-1, idx, dist))
            
        else :
            dist = calc_cos_dist(vectorized.getrow(num_docs-1), vectorized.getrow(idx))
            dists.append((num_docs-1, idx, dist))
        
    return dists

def sort_dists(dists):
    result_list = []
    for i, j, dist in sorted(dists, key=operator.itemgetter(2)):
        result_list.append((i, j, dist))
    return result_list

##### Text Preprocessing

In [6]:
# get data
article_df, comment_df = get_datas("2016-07-06")
len(article_df), len(comment_df)

(2241, 15)

In [57]:
# make documents
comment_str = comment_df["content"][:].str.cat(sep=' ')
comment_str

'영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 우선 오늘 저녁8부터는 다시 항공편 운항 재개를 한다고 하니 조금만 더 기다리시면 될 것 같아요. 제주도 관계자분들도 힘내세요! 영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 아울러 대구 지하철 참사를 다시 생각하게 해주셔서 감사합니다. 제주도를 비롯해서 다른 지역 충청이남서해안 지역분들과 전라도 부근 주민들도 힘내세요! 눈 때문에 고생이 많으세요! 그래도 자수를 했다니 다행이네요, 피해자분들도 마음 잘 추수리셔서 화이팅하셨으면 좋겠습니다. 경찰분들도 바쁘신건 알지만 힘 내주세요! 충북 제천지역도 다른지역과 같이 많이 추운가 봅니다ㅠ 농민분들이 엄청 고생하시는 것 같습니다. 조금만 있으면 날씨가 풀릴 것 같으니 힘내세요! 평창올림픽의 성공을 위하여 관련 관계자분들이 스페인 마드리드까지 가서 신다고 하는데 감사합니다. 평창올림픽의 성공적인 개최를 기원합니다. KTX 지연, 제주공항 운항 지연에 뒤이은 여러번의 동파사고까지, 조금만 지나면 날시가 풀릴테니 힘내세요!! 모쪼록 몸에 해로운 담배입니다. 다른 나라의 광고가 어떻든 대한민국 국민의 건강 증진을 위하여 금연을 기원하시는 분들은 금연에 성공하시길 바라겠습니다. 어린 아이들이 이렇게 모범을 보여주니 너무 고맙습니다. 화단정리에 노인들께 식사를 대접하기까지 너무 마음이 예쁩니다. 이렇게 노인장기요양서비스가 통합(통일)된다면 노인분들이 이용하시기에도 간편하고 좋은 서비스활동이 될 것 같습니다. 미래를 위해 방학중에 대학 공동 진로진학 박람회까지 가서 자신의 미래를 설꼐하는것을 보면 너무 아름다운 것 같습니다. 이 추운 날씨에도 고생해주시는 장병분들을 보면 감사한 마음 뿐입니다. 급식을 비롯해서 여러 여건들이 개선되길 희망합니다! 이렇게 추운날씨에 남들을 위해  쌀 1000kg을 기부해주신 대한불교조계종 문수사(주지 스님 혜안)께 너무 감사합니다! 이렇게

In [59]:
article_series = article_df["content"][2000:2240]
article_series[:5], article_series[-5:]

(2000    [뉴스데스크]◀ 앵커 ▶옷차림이 가벼워지면서 여성들이 신경 쓰는 것 중 하나가 제모...
 2001    [헤럴드경제=김진원ㆍ유오상 기자] 검찰이 유사수신 혐의로 수사를 받는 밸류인베스트코...
 2002    <앵커 멘트> 북한은 대내외적으로 잔뜩 전쟁 분위기를 띄우고 있습니다.   유치원생...
 2003    석탄화력발전 대책회의 모두발언    (서울=연합뉴스) 정하종 기자 = 주형환 산업통...
 2004    <앵커>초고층 건물이 밀집해 있는 부산에서는 지진에 놀란 시민들의 신고가 폭주했습니...
 Name: content, dtype: object,
 2235    【서울=뉴시스】유자비 기자 = 독일 폭스바겐 본사 고위 임원이 최근 한국을 방문한 ...
 2236    틀니와 임플란트이달부터 65세 이상에 건보 혜택임플란트, 평생 2개까지 적용돼치아 ...
 2237    숙련된 인력과 값싼 임금으로 지난 10여년 간 세계 정보기술(IT) 아웃소싱 강자로...
 2238    <앵커>최근 들어서 변호사가 그야말로 쏟아져 나오면서 대표적인 고소득 전문직이라는 ...
 2239    【서울=뉴시스】전진환 기자 = 박근혜 대통령이 6일 오후 2016 리우하계올림픽 D...
 Name: content, dtype: object)

In [60]:
article_series = article_series.append(pd.Series([comment_str])).reset_index(drop=True)
article_series[:5], article_series[-5:]

(0    [뉴스데스크]◀ 앵커 ▶옷차림이 가벼워지면서 여성들이 신경 쓰는 것 중 하나가 제모...
 1    [헤럴드경제=김진원ㆍ유오상 기자] 검찰이 유사수신 혐의로 수사를 받는 밸류인베스트코...
 2    <앵커 멘트> 북한은 대내외적으로 잔뜩 전쟁 분위기를 띄우고 있습니다.   유치원생...
 3    석탄화력발전 대책회의 모두발언    (서울=연합뉴스) 정하종 기자 = 주형환 산업통...
 4    <앵커>초고층 건물이 밀집해 있는 부산에서는 지진에 놀란 시민들의 신고가 폭주했습니...
 dtype: object, 236    틀니와 임플란트이달부터 65세 이상에 건보 혜택임플란트, 평생 2개까지 적용돼치아 ...
 237    숙련된 인력과 값싼 임금으로 지난 10여년 간 세계 정보기술(IT) 아웃소싱 강자로...
 238    <앵커>최근 들어서 변호사가 그야말로 쏟아져 나오면서 대표적인 고소득 전문직이라는 ...
 239    【서울=뉴시스】전진환 기자 = 박근혜 대통령이 6일 오후 2016 리우하계올림픽 D...
 240    영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연...
 dtype: object)

In [61]:
%time article_documents = make_document_morpheme(article_series)

CPU times: user 1min 47s, sys: 516 ms, total: 1min 47s
Wall time: 31.9 s


In [69]:
# make gen documents
gen_articles = gen_text_documents(article_documents)
gen_articles[0]

'요즘 금지 구매 사용 병원 화상 판매 예전 인터넷 의료 경우 경우 가정 저작권 모낭 시력 피부 피부 피부 피부 피부 피부 보습제 염증 색소 색소 배포 나중 신경 허가 진정 노출 노출 노출 노출 식품의약품안전처 은주 흡수 햇볕 수영복 앵커 털이 털이 제모 제모 제모 제모 제모 제모 방식 최소 표시 휴가철 리포트 중심 동안 설명 확인 보상 시술 기자 기자 손상 주의 침착 침착 모기 모기 모기 모기 옷차림 MBC뉴스 복제 우려 피해 주위 뉴스데스크 전문의 번호 무단 원리 백화점 여름 여름 팔다리 팔다리 자외선 레이저 레이저 레이저 레이저 레이저 레이저 레이저 레이저 기기 햇빛 본인 당부 여성 다음 영향'

##### Vectorizer

In [63]:
# vectorizer = CountVectorizer()
# vectorizer = HashingVectorizer()
vectorizer = TfidfVectorizer()

vectorized = vectorizer.fit_transform(gen_articles)
num_docs, num_features =  vectorized.shape
feature_names = vectorizer.get_feature_names()

print("the number of input documents : {}".format(num_docs))
print("the number of words : {}".format(num_features))
result_df = pd.DataFrame(vectorized.toarray(), columns=feature_names)

the number of input documents : 241
the number of words : 8172


##### Measurement Distance between comment and document

In [64]:
dists = get_euc_dists(vectorized, num_docs)
result = sort_dists(dists)
result[:5], result[-5:]

([(240, 9, 1.3116234460703404),
  (240, 138, 1.3530893640786008),
  (240, 108, 1.3538578136087327),
  (240, 215, 1.3576447694648097),
  (240, 103, 1.3673355836344543)],
 [(240, 154, 1.4142135623730951),
  (240, 161, 1.4142135623730951),
  (240, 173, 1.4142135623730951),
  (240, 233, 1.4142135623730954),
  (240, 50, 1.4142135623730958)])

In [65]:
dists = get_euc_dists(vectorized, num_docs, "cos")
result = sort_dists(dists)
result[:5], result[-5:]

([(240, 9, 0.86017803214071742),
  (240, 138, 0.91542541359131624),
  (240, 108, 0.91646548973470854),
  (240, 215, 0.92159966002757809),
  (240, 103, 0.93480329913648685)],
 [(240, 84, 1.0),
  (240, 154, 1.0),
  (240, 161, 1.0),
  (240, 173, 1.0),
  (240, 233, 1.0)])

In [66]:
# comment
print(comment_str)

영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 우선 오늘 저녁8부터는 다시 항공편 운항 재개를 한다고 하니 조금만 더 기다리시면 될 것 같아요. 제주도 관계자분들도 힘내세요! 영화 `로봇소리`많이 기대하고 있습니다. 폭풍 연기력을 보여주실 이성민 배우분의 연기력을 기대해봅니다. 아울러 대구 지하철 참사를 다시 생각하게 해주셔서 감사합니다. 제주도를 비롯해서 다른 지역 충청이남서해안 지역분들과 전라도 부근 주민들도 힘내세요! 눈 때문에 고생이 많으세요! 그래도 자수를 했다니 다행이네요, 피해자분들도 마음 잘 추수리셔서 화이팅하셨으면 좋겠습니다. 경찰분들도 바쁘신건 알지만 힘 내주세요! 충북 제천지역도 다른지역과 같이 많이 추운가 봅니다ㅠ 농민분들이 엄청 고생하시는 것 같습니다. 조금만 있으면 날씨가 풀릴 것 같으니 힘내세요! 평창올림픽의 성공을 위하여 관련 관계자분들이 스페인 마드리드까지 가서 신다고 하는데 감사합니다. 평창올림픽의 성공적인 개최를 기원합니다. KTX 지연, 제주공항 운항 지연에 뒤이은 여러번의 동파사고까지, 조금만 지나면 날시가 풀릴테니 힘내세요!! 모쪼록 몸에 해로운 담배입니다. 다른 나라의 광고가 어떻든 대한민국 국민의 건강 증진을 위하여 금연을 기원하시는 분들은 금연에 성공하시길 바라겠습니다. 어린 아이들이 이렇게 모범을 보여주니 너무 고맙습니다. 화단정리에 노인들께 식사를 대접하기까지 너무 마음이 예쁩니다. 이렇게 노인장기요양서비스가 통합(통일)된다면 노인분들이 이용하시기에도 간편하고 좋은 서비스활동이 될 것 같습니다. 미래를 위해 방학중에 대학 공동 진로진학 박람회까지 가서 자신의 미래를 설꼐하는것을 보면 너무 아름다운 것 같습니다. 이 추운 날씨에도 고생해주시는 장병분들을 보면 감사한 마음 뿐입니다. 급식을 비롯해서 여러 여건들이 개선되길 희망합니다! 이렇게 추운날씨에 남들을 위해  쌀 1000kg을 기부해주신 대한불교조계종 문수사(주지 스님 혜안)께 너무 감사합니다! 이렇게 

##### Recomend Article

In [67]:
comment, rank_idx, distance = zip(*result)
idx = rank_idx[0]
print(idx)
print(article_series[idx].split("▶")[0])

9
로사 코른펠드-마테 [유엔 홈페이지 캡처]노인이 브렉시트 희생양…인권 전문가 "민주사회 근본 위협" 우려      (서울=연합뉴스) 한미희 기자 = 영국에서 유럽연합(EU) 탈퇴(브렉시트) 국민투표 이후 노인을 향한 차별적인 공격이 난무해 유엔이 경계하고 나섰다.     유엔인권최고대표사무소(OHCHR)가 임명한 노인 인권 전문가인 로사 코른펠드-마테는 5일(현지시간) 유엔 홈페이지를 통해 "유럽의 많은 언론과 소셜미디어가 노인을 브렉시트의 희생양으로 삼고 이들의 투표권을 제한해야 한다고 요구하고 있다"고 전했다.     그러면서 "나이를 근거로 특정 권리의 행사를 제한하는 것은 국제 인권법상 용납할 수 없는 일"이라고 강조했다. 지난달 23일 브렉시트 투표소로 향하는 여성 노인 [AFP=연합뉴스]    지난달 23일 치러진 영국의 브렉시트 국민투표에서 65세 이상 노인들은 대거 탈퇴를 지지해 잔류를 원했던 젊은 EU 세대와의 극명한 격차를 보였다.     투표 결과에 대해 젊은이들은 '노인들이 우리의 미래를 망쳤다'고 반발하며 세대 갈등을 드러냈다.     코른펠드-마테는 "한 잡지는 일정 연령 이상이 되면 노인들의 운전면허를 박탈하듯 투표권을 빼앗아야 한다고 주장하고 있다"고 단적인 사례를 소개했다.     그는 "나이가 들었다는 이유로 그 사람의 정치적 권한에 이의를 제기해서는 안 된다"며 "공적 영역과 의사 결정 과정에서 모든 개인의 동등한 권리를 존중하는 것은 민주사회의 근본"이라고 말했다.     그러면서 "전 세계의 사회가 고령화하면서 세대 간 결속과 노인의 가치 있는 기여에 대한 사회적 이해를 높이기 위해 투자가 필요하다"고 목소리를 높였다. 제복입은 노인들의 투표행진(AFP=연합뉴스)     mihee@yna.co.kr


In [68]:
idx = len(rank_idx) - 1
print(article_series[idx].split("▶")[0])

【서울=뉴시스】전진환 기자 = 박근혜 대통령이 6일 오후 2016 리우하계올림픽 D-30 선수단 격려를 위해 서울 노원구 태릉선수촌을 찾아, 여자핸드볼 국가대표팀선수들을 격려하고 있다. 2016.07.06.   amin2@newsis.com태릉선수촌 방문…리우올림픽 선수단 격려  "질병·테러 철저히 대비해 선수단 안전에 만전 기해야"【서울=뉴시스】김형섭 기자 = 박근혜 대통령은 6일 브라질 리우하계올림픽과 관련해 "저도 여기서 태극전사들이 선전하도록 열심히 응원하겠다"고 말했다. 박 대통령은 브라질 리우하계올림픽을 30일 앞둔 이날 오후 국가대표 선수단 및 지도자 격려차 서울 노원구 태릉선수촌을 방문한 자리에서 "우리 선수들이 최상의 컨디션으로 경기에 임할 수 있도록 끝까지 잘 챙겨주시고 판정시비라든지 현지에서 힘든 일들로 (고생을 겪기 않기를) 바라겠다"면서 이같이 밝혔다. 박 대통령의 태릉선수촌 방문은 2014년 8월25일 인천아시안게임 국가대표 선수단 격려를 위한 방문 이후 약 2년 만이다. 특히 박 대통령은 이날 최종삼 태릉선수촌장으로부터 리우올림픽 준비 상황을 보고 받은 뒤 "올림픽 개최지인 브라질의 열악한 환경 등을 고려해 선수 컨디션 조절 및 사전 적응훈련 등 대회 준비에 최선을 다해달라"며 "질병, 테러 등에도 철저히 대비해 선수단 안전에 만전을 기해달라"고 강조했다고 청와대가 전했다. 이어 박 대통령은 개선관과 월계관을 찾아 태권도, 펜싱, 체조, 역도, 레슬링, 핸드볼, 유도 종목 선수들의 훈련을 참관했다. 박 대통령은 선수들을 직접 만나 격려하고 그 동안 흘린 땀방울이 결실을 맺을 수 있도록 리우하계올림픽에서 최선을 다해 줄 것을 주문했다.【서울=뉴시스】전진환 기자 = 박근혜 대통령이 6일 오후 2016 리우하계올림픽 D-30 선수단 격려를 위해 서울 노원구 태릉선수촌을 찾아, 체조대표팀선수들과 기념촬영을 하고 있다. 2016.07.06.   amin2@newsis.com 태권도장을 방문한 자리에서 박 대통령은 "순방으로 여러 나라를 가보면

### 추가할 사항

- 네이버 기사 카테고리 분류 모델로 댓글 마다 카테고리를 분류해서 카테고리에 대한 수치도 기사 추천 수치에 추가
- 추천 결과에 대한 검증방법이 있는지?, 